In [1]:
import numpy as np
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import math

# Read Data

In [2]:
import os
#@kaavya update directory
directory = 'transcripts'

list_csv = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if f[-3:] == 'csv':
            list_csv += [f]
import re
all_transcript_files = [re.search('([^\/]+$)', list_csv[i])[0] for i in range(len(list_csv))]
all_transcript_files

['rough_translation_transcripts.csv',
 'embedded_transcripts.csv',
 'pop_culture_happy_hour_transcripts.csv',
 'here_and_now_transcripts.csv',
 'its_been_a_minute_transcripts.csv',
 'npr_politics_podcast_transcripts.csv',
 'bullseye_with_jesse_thorn_transcripts.csv',
 'invisibilia_transcripts.csv',
 'ted_radio_hour_transcripts.csv',
 'throughline_transcripts.csv',
 'planet_money_transcripts.csv',
 'short_wave_transcripts.csv',
 'code_switch_transcripts.csv',
 'indicator_from_planet_money_transcripts.csv',
 'fresh_air_transcripts.csv',
 'life_kit_transcripts.csv',
 '1A_transcripts.csv',
 'consider_this_transcripts.csv']

In [3]:
import pandas as pd
for i in range(len(all_transcript_files)):
    if i == 0:
        df = pd.read_csv('transcripts/'+ all_transcript_files[i])
    else:
        df = pd.concat([df, pd.read_csv('transcripts/'+ all_transcript_files[i], index_col = 0)])
all_transcripts = df.drop(columns=['Unnamed: 0'])
all_transcripts

,title,link,description,duration,podcast_id,transcript
0,Letter of Unhappiness,https://www.npr.org/2022/03/30/1089668294/lett...,When Naira calls her parents back home in Russ...,950.0,1089668294,"[<div aria-label=""Transcript"" class=""transcrip..."
1,The Culture Front,https://www.npr.org/2022/03/15/1086679997/the-...,When protecting a language is used as justific...,2436.0,1086679997,"[<div aria-label=""Transcript"" class=""transcrip..."
2,Fighting Words In Ukraine,https://www.npr.org/2022/03/02/1083960666/figh...,Vladimir Putin joined the KGB at age 23. Ukrai...,1883.0,1083960666,"[<div aria-label=""Transcript"" class=""transcrip..."
3,"Presenting 'TED Radio Hour': Work, Play, Rest",https://www.npr.org/2022/02/15/1080834112/pres...,The past few years have shaken the fundamental...,3040.0,1080834112,NaN
4,May We Have This Dance?,https://www.npr.org/2021/12/22/1066965712/may-...,A jazz dance born in Harlem in the 1920s ends ...,2630.0,1066965712,"[<div aria-label=""Transcript"" class=""transcrip..."
...,...,...,...,...,...,...
598,"New York City, U.S. Epicenter, Braces For Peak",https://www.npr.org/2020/03/24/820610818/new-y...,Governor Andrew Cuomo said the pandemic could ...,750.0,820610818,"[<div aria-label=""Transcript"" class=""transcrip..."
599,The Cost Of Social Distancing,https://www.npr.org/2020/03/23/820040133/the-c...,How do officials weigh the economic cost again...,746.0,820040133,"[<div aria-label=""Transcript"" class=""transcrip..."
600,"CA, NY On Lockdown; Mortgage Relief For Some H...",https://www.npr.org/2020/03/20/819051536/ca-ny...,Two of the hardest-hit states order residents ...,706.0,819051536,"[<div aria-label=""Transcript"" class=""transcrip..."
601,GOP Senator Raised Virus Alarms Weeks Ago — In...,https://www.npr.org/2020/03/19/818243084/gop-s...,"U.S. Sen. Richard Burr, in a private luncheon,...",774.0,818243084,"[<div aria-label=""Transcript"" class=""transcrip..."


In [4]:
#all_transcripts = pd.concat([politics, code_switch, planet_money, pop_culture, throughline], axis=0)
all_transcripts = all_transcripts[all_transcripts['transcript'].notnull()].reset_index(drop=True)
all_transcripts

,title,link,description,duration,podcast_id,transcript
0,Letter of Unhappiness,https://www.npr.org/2022/03/30/1089668294/lett...,When Naira calls her parents back home in Russ...,950.0,1089668294,"[<div aria-label=""Transcript"" class=""transcrip..."
1,The Culture Front,https://www.npr.org/2022/03/15/1086679997/the-...,When protecting a language is used as justific...,2436.0,1086679997,"[<div aria-label=""Transcript"" class=""transcrip..."
2,Fighting Words In Ukraine,https://www.npr.org/2022/03/02/1083960666/figh...,Vladimir Putin joined the KGB at age 23. Ukrai...,1883.0,1083960666,"[<div aria-label=""Transcript"" class=""transcrip..."
3,May We Have This Dance?,https://www.npr.org/2021/12/22/1066965712/may-...,A jazz dance born in Harlem in the 1920s ends ...,2630.0,1066965712,"[<div aria-label=""Transcript"" class=""transcrip..."
4,Moms In Translation,https://www.npr.org/2021/12/14/1064250980/moms...,An Irish journalist discovers she belongs in a...,2519.0,1064250980,"[<div aria-label=""Transcript"" class=""transcrip..."
...,...,...,...,...,...,...
4219,"New York City, U.S. Epicenter, Braces For Peak",https://www.npr.org/2020/03/24/820610818/new-y...,Governor Andrew Cuomo said the pandemic could ...,750.0,820610818,"[<div aria-label=""Transcript"" class=""transcrip..."
4220,The Cost Of Social Distancing,https://www.npr.org/2020/03/23/820040133/the-c...,How do officials weigh the economic cost again...,746.0,820040133,"[<div aria-label=""Transcript"" class=""transcrip..."
4221,"CA, NY On Lockdown; Mortgage Relief For Some H...",https://www.npr.org/2020/03/20/819051536/ca-ny...,Two of the hardest-hit states order residents ...,706.0,819051536,"[<div aria-label=""Transcript"" class=""transcrip..."
4222,GOP Senator Raised Virus Alarms Weeks Ago — In...,https://www.npr.org/2020/03/19/818243084/gop-s...,"U.S. Sen. Richard Burr, in a private luncheon,...",774.0,818243084,"[<div aria-label=""Transcript"" class=""transcrip..."


# Clean Data

In [5]:
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    #get rid of copyright
    raw_html = raw_html[1:-621]
    #get rid of html tags
    cleantext = re.sub(CLEANR, '', raw_html)
    cleantext = cleantext.replace("\\", "").replace("APPLAUSE", "").replace("(LAUGHTER)", "").replace('(laughter)', "").replace("(Laughter)", "")
    #get rid of people names
    cleantext = re.sub( r"([A-Z \\',#\d-]+:)",'',cleantext)
    #get rid of all soundbites
    cleantext = re.sub(r'\(SOUNDBITE.*?"\)|(\(SOUNDBITE [A-Z ]+\))', '', cleantext)
    #cleantext = re.sub(r'Copyright ©.*audio record\.', '', cleantext)
    cleantext = cleantext.replace('\n', '')    
    return cleantext

In [6]:
#For Testing Individual Scripts

#test = all_transcripts['transcript'][1333][1:-621]
#cleanhtml(test)

In [7]:
all_transcripts['cleaned_transcripts'] = all_transcripts['transcript'].apply(cleanhtml)
all_transcripts

,title,link,description,duration,podcast_id,transcript,cleaned_transcripts
0,Letter of Unhappiness,https://www.npr.org/2022/03/30/1089668294/lett...,When Naira calls her parents back home in Russ...,950.0,1089668294,"[<div aria-label=""Transcript"" class=""transcrip...",You're listening to ROUGH TRANSLATION from N...
1,The Culture Front,https://www.npr.org/2022/03/15/1086679997/the-...,When protecting a language is used as justific...,2436.0,1086679997,"[<div aria-label=""Transcript"" class=""transcrip...",You're listening to ROUGH TRANSLATION from N...
2,Fighting Words In Ukraine,https://www.npr.org/2022/03/02/1083960666/figh...,Vladimir Putin joined the KGB at age 23. Ukrai...,1883.0,1083960666,"[<div aria-label=""Transcript"" class=""transcrip...",You're listening to ROUGH TRANSLATION from N...
3,May We Have This Dance?,https://www.npr.org/2021/12/22/1066965712/may-...,A jazz dance born in Harlem in the 1920s ends ...,2630.0,1066965712,"[<div aria-label=""Transcript"" class=""transcrip...",This is ROUGH TRANSLATION from NPR. I grew u...
4,Moms In Translation,https://www.npr.org/2021/12/14/1064250980/moms...,An Irish journalist discovers she belongs in a...,2519.0,1064250980,"[<div aria-label=""Transcript"" class=""transcrip...",Hey. It's Gregory from ROUGH TRANSLATION. Ju...
...,...,...,...,...,...,...,...
4219,"New York City, U.S. Epicenter, Braces For Peak",https://www.npr.org/2020/03/24/820610818/new-y...,Governor Andrew Cuomo said the pandemic could ...,750.0,820610818,"[<div aria-label=""Transcript"" class=""transcrip...","There are now more than 51,000 confirmed cas..."
4220,The Cost Of Social Distancing,https://www.npr.org/2020/03/23/820040133/the-c...,How do officials weigh the economic cost again...,746.0,820040133,"[<div aria-label=""Transcript"" class=""transcrip...","For many of us in the United States, this is..."
4221,"CA, NY On Lockdown; Mortgage Relief For Some H...",https://www.npr.org/2020/03/20/819051536/ca-ny...,Two of the hardest-hit states order residents ...,706.0,819051536,"[<div aria-label=""Transcript"" class=""transcrip...","At the White House today, the nation's top i..."
4222,GOP Senator Raised Virus Alarms Weeks Ago — In...,https://www.npr.org/2020/03/19/818243084/gop-s...,"U.S. Sen. Richard Burr, in a private luncheon,...",774.0,818243084,"[<div aria-label=""Transcript"" class=""transcrip...","There are now over 10,000 confirmed cases of..."


In [21]:
all_transcripts['cleaned_transcripts'][980]

' Hi. My name is Rotem (ph) from Pittsburgh, Pa. Our local makerspace has reopened, and after almost a year of watching blacksmithing videos online, I\'m about to pull a 2,000-degree piece of metal out of the forge for the first time. This podcast was recorded at... 07 p.m. Eastern Time on Tuesday, June 22. Things may have changed by the time you hear this. But hopefully, this old car part will be a brand-new knife. Enjoy the show. Some great sound there. Some good ambi (ph) - that\'s a radio-quality ambi sound . Hey there. It\'s the NPR POLITICS PODCAST. I\'m Asma Khalid. I cover the White House. I\'m Susan Davis. I cover Congress. And I\'m Mara Liasson, national political correspondent. Democrats have been pushing this sweeping voting rights bill to overhaul elections, and it\'s all but guaranteed to fall apart in the Senate today. The bill would create new federal mandates to make it easier to vote, toughen campaign finance disclosure laws and enact new ethics requirements on every 

# Clean Descriptions

In [22]:
LEANR = re.compile('<.*?>') 

def clean_description(raw_html):
    #get rid of html tags
    cleantext = re.sub(CLEANR, '', raw_html)
    #get rid of npr message at end of summary
    cleantext = re.sub(r'This episode.+', '', cleantext)
    return cleantext

In [23]:
all_transcripts['cleaned_description'] = all_transcripts['description'].apply(clean_description)
all_transcripts

,title,link,description,duration,podcast_id,transcript,cleaned_transcripts,cleaned_description
0,Letter of Unhappiness,https://www.npr.org/2022/03/30/1089668294/lett...,When Naira calls her parents back home in Russ...,950.0,1089668294,"[<div aria-label=""Transcript"" class=""transcrip...",You're listening to ROUGH TRANSLATION from N...,When Naira calls her parents back home in Russ...
1,The Culture Front,https://www.npr.org/2022/03/15/1086679997/the-...,When protecting a language is used as justific...,2436.0,1086679997,"[<div aria-label=""Transcript"" class=""transcrip...",You're listening to ROUGH TRANSLATION from N...,When protecting a language is used as justific...
2,Fighting Words In Ukraine,https://www.npr.org/2022/03/02/1083960666/figh...,Vladimir Putin joined the KGB at age 23. Ukrai...,1883.0,1083960666,"[<div aria-label=""Transcript"" class=""transcrip...",You're listening to ROUGH TRANSLATION from N...,Vladimir Putin joined the KGB at age 23. Ukrai...
3,May We Have This Dance?,https://www.npr.org/2021/12/22/1066965712/may-...,A jazz dance born in Harlem in the 1920s ends ...,2630.0,1066965712,"[<div aria-label=""Transcript"" class=""transcrip...",This is ROUGH TRANSLATION from NPR. I grew u...,A jazz dance born in Harlem in the 1920s ends ...
4,Moms In Translation,https://www.npr.org/2021/12/14/1064250980/moms...,An Irish journalist discovers she belongs in a...,2519.0,1064250980,"[<div aria-label=""Transcript"" class=""transcrip...",Hey. It's Gregory from ROUGH TRANSLATION. Ju...,An Irish journalist discovers she belongs in a...
...,...,...,...,...,...,...,...,...
4219,"New York City, U.S. Epicenter, Braces For Peak",https://www.npr.org/2020/03/24/820610818/new-y...,Governor Andrew Cuomo said the pandemic could ...,750.0,820610818,"[<div aria-label=""Transcript"" class=""transcrip...","There are now more than 51,000 confirmed cas...",Governor Andrew Cuomo said the pandemic could ...
4220,The Cost Of Social Distancing,https://www.npr.org/2020/03/23/820040133/the-c...,How do officials weigh the economic cost again...,746.0,820040133,"[<div aria-label=""Transcript"" class=""transcrip...","For many of us in the United States, this is...",How do officials weigh the economic cost again...
4221,"CA, NY On Lockdown; Mortgage Relief For Some H...",https://www.npr.org/2020/03/20/819051536/ca-ny...,Two of the hardest-hit states order residents ...,706.0,819051536,"[<div aria-label=""Transcript"" class=""transcrip...","At the White House today, the nation's top i...",Two of the hardest-hit states order residents ...
4222,GOP Senator Raised Virus Alarms Weeks Ago — In...,https://www.npr.org/2020/03/19/818243084/gop-s...,"U.S. Sen. Richard Burr, in a private luncheon,...",774.0,818243084,"[<div aria-label=""Transcript"" class=""transcrip...","There are now over 10,000 confirmed cases of...","U.S. Sen. Richard Burr, in a private luncheon,..."


In [24]:
# all_transcripts.to_csv('cleaned_transcripts_v2.csv')